In [3]:
#Carga de Librerias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [4]:
begin_inv_2016 = pd.read_csv("BegInvFINAL12312016.csv")

In [5]:
#informacion del dataframe

begin_inv_2016.head(), begin_inv_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  206529 non-null  object 
 1   Store        206529 non-null  int64  
 2   City         206529 non-null  object 
 3   Brand        206529 non-null  int64  
 4   Description  206529 non-null  object 
 5   Size         206529 non-null  object 
 6   onHand       206529 non-null  int64  
 7   Price        206529 non-null  float64
 8   startDate    206529 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 14.2+ MB


(         InventoryId  Store          City  Brand                  Description  \
 0  1_HARDERSFIELD_58      1  HARDERSFIELD     58  Gekkeikan Black & Gold Sake   
 1  1_HARDERSFIELD_60      1  HARDERSFIELD     60       Canadian Club 1858 VAP   
 2  1_HARDERSFIELD_62      1  HARDERSFIELD     62     Herradura Silver Tequila   
 3  1_HARDERSFIELD_63      1  HARDERSFIELD     63   Herradura Reposado Tequila   
 4  1_HARDERSFIELD_72      1  HARDERSFIELD     72         No. 3 London Dry Gin   
 
     Size  onHand  Price   startDate  
 0  750mL       8  12.99  2016-01-01  
 1  750mL       7  10.99  2016-01-01  
 2  750mL       6  36.99  2016-01-01  
 3  750mL       3  38.99  2016-01-01  
 4  750mL       6  34.99  2016-01-01  ,
 None)

In [35]:
#Estadisticas de cada columna
for i in begin_inv_2016.columns:
    print(i)
    #Esto imprime las estadísticas descriptivas (como count, mean, std, min, 25%, 50%, 75%, max) para esa columna.
    print(begin_inv_2016[i].describe())
    print('--------------------------------------------------------------')

InventoryId
count                 202806
unique                202806
top       79_BALLYMENA_90089
freq                       1
Name: InventoryId, dtype: object
--------------------------------------------------------------
Store
count    202806.000000
mean         42.146638
std          23.181687
min           1.000000
25%          22.000000
50%          42.000000
75%          64.000000
max          79.000000
Name: Store, dtype: float64
--------------------------------------------------------------
City
count        202806
unique           67
top       MOUNTMEND
freq          14075
Name: City, dtype: object
--------------------------------------------------------------
Brand
count    202806.000000
mean      13966.557814
std       13064.421745
min          58.000000
25%        3802.000000
50%        8096.000000
75%       22508.750000
max       90090.000000
Name: Brand, dtype: float64
--------------------------------------------------------------
Description
count                   2028

In [7]:
#Convertir la columna StartDate en tipo fecha

begin_inv_2016['startDate'] = pd.to_datetime(begin_inv_2016['startDate'])


In [9]:
#Exploración de Valores Anómalos
#onHand y Price verificar que estas columnas no tengan valores negativos, 
# ya que onHand debería ser positivo (o cero) y Price generalmente también.

begin_inv_2016 = begin_inv_2016[begin_inv_2016['onHand'] >= 0]
begin_inv_2016 = begin_inv_2016[begin_inv_2016['Price'] >= 0]

In [34]:
print(begin_inv_2016['Size_ml'])

0         750.0
1         750.0
2         750.0
3         750.0
4         750.0
          ...  
206524    750.0
206525    750.0
206526    750.0
206527    750.0
206528    750.0
Name: Size_ml, Length: 202806, dtype: float64


In [19]:
import re

def convertir_a_ml(size):
    # Expresión regular para extraer el número principal, unidad y posibles cantidades de paquetes
    match = re.match(r"(\d+(\.\d+)?)\s*(mL|L|liter|oz)?(?:\s*(\d+)?\s*Pk)?", str(size), re.IGNORECASE)
    
    if match:
        valor = float(match.group(1))  # Valor numérico principal
        unidad = match.group(3).lower() if match.group(3) else ''  # Unidad de medida
        cantidad_pk = int(match.group(4)) if match.group(4) else 1  # Cantidad en el paquete, por defecto 1
        
        # Convertir a mililitros según la unidad
        if unidad in ['l', 'liter']:
            return valor * 1000 * cantidad_pk  # Convertir litros a mililitros y multiplicar por cantidad
        elif unidad == 'oz':
            return valor * 29.5735 * cantidad_pk  # Convertir onzas a mililitros y multiplicar por cantidad
        elif unidad == 'ml':
            return valor * cantidad_pk  # Ya está en mililitros, multiplicar por cantidad
    return None

# Aplicar la función a la columna 'Size' para crear la columna 'Size_ml' en mL
begin_inv_2016['Size_ml'] = begin_inv_2016['Size'].apply(convertir_a_ml)

# Verificar el resultado
print(begin_inv_2016[['Size', 'Size_ml']].head(10))


    Size  Size_ml
0  750mL    750.0
1  750mL    750.0
2  750mL    750.0
3  750mL    750.0
4  750mL    750.0
5  750mL    750.0
6  750mL    750.0
7  750mL    750.0
8  Liter      NaN
9  750mL    750.0


In [31]:
# Eliminar filas donde 'Size_ml' tiene valores NaN
begin_inv_2016 = begin_inv_2016.dropna(subset=['Size_ml'])

# Verificar el resultado
print(begin_inv_2016['Size_ml'].unique())


[  750.      1750.        50.       375.       200.      2250.
  1125.       600.       150.       400.       100.       250.
   800.      1000.      1500.      5000.      4000.       748.
  3000.       147.8675   187.       500.       561.       300.
   330.     18000.       180.       720.       650.617   6000.
 20000.    ]


In [36]:
#Borrar la columna Size para que solo quede Size_ml
begin_inv_2016.drop('Size', axis=1)

,InventoryId,Store,City,Brand,Description,onHand,Price,startDate,Size_ml
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,8,12.99,2016-01-01,750.0
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,7,10.99,2016-01-01,750.0
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,6,36.99,2016-01-01,750.0
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,3,38.99,2016-01-01,750.0
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,6,34.99,2016-01-01,750.0
...,...,...,...,...,...,...,...,...,...
206524,79_BALLYMENA_46985,79,BALLYMENA,46985,Rodney Strong Cab Svgn Alexa,13,22.99,2016-01-01,750.0
206525,79_BALLYMENA_47014,79,BALLYMENA,47014,Juan Gil Jumilla Rd,13,13.99,2016-01-01,750.0
206526,79_BALLYMENA_47090,79,BALLYMENA,47090,Napa Cellars Cab Svgn Napa,19,23.99,2016-01-01,750.0
206527,79_BALLYMENA_90011,79,BALLYMENA,90011,Ch Pichon Longville 12 Pauil,12,144.99,2016-01-01,750.0


In [37]:
begin_inv_2016.to_csv("Begin_inv_2016.csv")